In [23]:
from transitions import Machine

from transitions.extensions.asyncio import AsyncTimeout, AsyncMachine

import enum 
import logging
import asyncio

from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
from pprint import pprint

from datetime import datetime
from math import floor, ceil
import time
from time import sleep

import sys
import os

import zmq
import zmq.asyncio
# from s import Context
import json

logging.basicConfig(level=logging.INFO)

import ccxt
from dotenv import load_dotenv
load_dotenv()


True

In [24]:
client = MongoClient()
db=client.fsmbot
# trades = {}
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})



In [39]:
# exchange.fetchOrderStatus('686722117', symbol='ALGOUSDT')
exchange.fetchOpenOrders(symbol='ALGOUSDT')

[{'info': {'symbol': 'ALGOUSDT',
   'orderId': '686886090',
   'orderListId': '-1',
   'clientOrderId': 'zenk7KI8LCXLkLdQuVBMIU',
   'price': '1.59620000',
   'origQty': '14.00000000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'STOP_LOSS_LIMIT',
   'side': 'SELL',
   'stopPrice': '1.77350000',
   'icebergQty': '0.00000000',
   'time': '1633577559238',
   'updateTime': '1633577559238',
   'isWorking': False,
   'origQuoteOrderQty': '0.00000000'},
  'id': '686886090',
  'clientOrderId': 'zenk7KI8LCXLkLdQuVBMIU',
  'timestamp': 1633577559238,
  'datetime': '2021-10-07T03:32:39.238Z',
  'lastTradeTimestamp': None,
  'symbol': 'ALGO/USDT',
  'type': 'stop_loss_limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'sell',
  'price': 1.5962,
  'stopPrice': 1.7735,
  'amount': 14.0,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 14.0,
  'status': 'open',
  'fee': None,
  'trades': 

In [36]:
exchange.fetchOrderStatus('686722117', symbol='ALGOUSDT')
# exchange.fetchOrders(symbol='ALGOUSDT')

'closed'

In [8]:
def buyOrder(symbol, status, qty, trigger, stop, target):
    datecreated = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    db.trades.insert_many([
        {
            "id": f"{symbol}_{datecreated}",
            "datecreated" : datecreated,
            "state": 'LIVE',
            "symbol" : symbol,
            "trigger": trigger,
            "stop": stop,
            "target": target,
            "qty": qty
        }
    ]);

def sellOrder(symbol, status, qty, target):
    datecreated = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    db.trades.insert_many([
        {
            "id": f"{symbol}_{datecreated}",
            "datecreated" : datecreated,
            "state": status,
            "symbol" : symbol,
            "target": target,
            "qty": qty
        }
    ]);    

In [48]:
status = "new"
size = 15
symbol = "ADAUSDT"
buyTarget = "2.2"
buyStop = "38.00"
sellTarget = "3"
datecreated = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

trades.insert_many([
    {
        "id": f"{symbol}_{datecreated}",
        "datecreated" : datecreated,
        "status": status,
        "symbol" : symbol,
        "buyTarget": buyTarget,
        "buyStop": buyStop,
        "sellTarget": sellTarget,
        "size": size
    }
])

In [5]:
capital = 50000.0
risk = 0.01

symbol = "AVAXUSDT"
trigger = 65.35
stop = 63.68
target = 73.16
# (capital * risk) = (trigger - stop) * qty
qty = (capital * risk) / (trigger - stop)
size = qty * trigger
# buyOrder(symbol, "buyat", size, stop, target)
qty, size

(299.4011976047914, 19565.868263473116)

In [7]:
(capital, risk) = 50000, 0.01
symbol = 'ALGOUSDT'
(trigger, stop, target) = 1.8930, 1.8533, 1.9837
qty = floor( (capital * risk) / (trigger - stop) )
size = qty * trigger
buyOrder(symbol, "buyat", qty, trigger, stop, target)
qty, size

(12594, 23840.442)

In [29]:
# test ticker zmq publisher
# zContext = zmq.asyncio.Context()

socket = zContext.socket(zmq.SUB)
socket.connect("tcp://localhost:5556")
socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

while True:
    tickerString = await socket.recv_string()
    _, symbol, ticker = tickerString.split()
    ticker = json.loads(ticker)
    if symbol == 'BTCUSDT':
        print(symbol)


BTCUSDT
BTCUSDT
BTCUSDT
BTCUSDT
BTCUSDT


CancelledError: 

In [16]:
try:
    raise Exception('BOOOOM')
except BaseException as err:
    print(err)


BOOOOM
